In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')


%cd '/content/drive/MyDrive/Project/'
%pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1XSIbHWFizvrGifGPTwtsmagK2AS8ZE7n/Project


'/content/drive/.shortcut-targets-by-id/1XSIbHWFizvrGifGPTwtsmagK2AS8ZE7n/Project'

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [ ]:
np.random.seed(682)
tf.random.set_seed(682)

In [ ]:
class Distiller(keras.Model):

    def __init__(self, student1, student2, student3, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student1 = student1
        self.student2 = student2
        self.student3 = student3

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        beta=0.8,
        gamma=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.temperature = temperature

    def train_step(self, data):
        
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape_student_1:
            
            # Forward pass of student
            student_predictions_1 = self.student1(x, training=True)
            student_predictions_2 = self.student2(x, training=False)
            student_predictions_3 = self.student3(x, training=False)

            # Compute losses
            student_loss_1 = self.student_loss_fn(y, student_predictions_1)
            student_loss_2 = self.student_loss_fn(y, student_predictions_2)
            student_loss_3 = self.student_loss_fn(y, student_predictions_3)

            # for student 1
            distillation_loss_t_s1 = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions_1 / self.temperature, axis=1)
            )
            distillation_loss_s1_s2 = self.distillation_loss_fn(
                tf.nn.softmax(student_predictions_1 / self.temperature, axis=1),
                tf.nn.softmax(student_predictions_2 / self.temperature, axis=1)
            )
            distillation_loss_s1_s3 = self.distillation_loss_fn(
                tf.nn.softmax(student_predictions_1 / self.temperature, axis=1),
                tf.nn.softmax(student_predictions_3 / self.temperature, axis=1)
            )

            loss_1 = self.alpha * student_loss_1 + self.beta * distillation_loss_t_s1 + self.gamma * (distillation_loss_s1_s2 + distillation_loss_s1_s3)

        # Compute gradients
        trainable_vars_1 = self.student1.trainable_variables
        gradients_1 = tape_student_1.gradient(loss_1, trainable_vars_1)
        self.optimizer.apply_gradients(zip(gradients_1, trainable_vars_1))
        self.compiled_metrics._metrics[0].update_state(y, student_predictions_1)
        

        with tf.GradientTape() as tape_student_2:
            
            # Forward pass of student
            student_predictions_1 = self.student1(x, training=False)
            student_predictions_2 = self.student2(x, training=True)
            student_predictions_3 = self.student3(x, training=False)

            # Compute losses
            student_loss_1 = self.student_loss_fn(y, student_predictions_1)
            student_loss_2 = self.student_loss_fn(y, student_predictions_2)
            student_loss_3 = self.student_loss_fn(y, student_predictions_3)
            
            # for student 2
            distillation_loss_t_s2 = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions_2 / self.temperature, axis=1)
            )
            distillation_loss_s2_s1 = self.distillation_loss_fn(
                tf.nn.softmax(student_predictions_2 / self.temperature, axis=1),
                tf.nn.softmax(student_predictions_1 / self.temperature, axis=1)
            )
            distillation_loss_s2_s3 = self.distillation_loss_fn(
                tf.nn.softmax(student_predictions_2 / self.temperature, axis=1),
                tf.nn.softmax(student_predictions_3 / self.temperature, axis=1)
            )

            loss_2 = self.alpha * student_loss_2 + self.beta * distillation_loss_t_s2 + self.gamma * (distillation_loss_s2_s1 + distillation_loss_s2_s3)

        trainable_vars_2 = self.student2.trainable_variables
        gradients_2 = tape_student_2.gradient(loss_2, trainable_vars_2)

        self.optimizer.apply_gradients(zip(gradients_2, trainable_vars_2))

        self.compiled_metrics._metrics[1].update_state(y, student_predictions_2)
            

        with tf.GradientTape() as tape_student_3:
            
            # Forward pass of student
            student_predictions_1 = self.student1(x, training=False)
            student_predictions_2 = self.student2(x, training=False)
            student_predictions_3 = self.student3(x, training=True)

            # Compute losses
            student_loss_1 = self.student_loss_fn(y, student_predictions_1)
            student_loss_2 = self.student_loss_fn(y, student_predictions_2)
            student_loss_3 = self.student_loss_fn(y, student_predictions_3)

            # for student 3
            distillation_loss_t_s3 = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions_3 / self.temperature, axis=1)
            )
            distillation_loss_s3_s1 = self.distillation_loss_fn(
                tf.nn.softmax(student_predictions_3 / self.temperature, axis=1),
                tf.nn.softmax(student_predictions_1 / self.temperature, axis=1)
            )
            distillation_loss_s3_s2 = self.distillation_loss_fn(
                tf.nn.softmax(student_predictions_3 / self.temperature, axis=1),
                tf.nn.softmax(student_predictions_2 / self.temperature, axis=1)
            )

            loss_3 = self.alpha * student_loss_3 + self.beta * distillation_loss_t_s3 + self.gamma * (distillation_loss_s3_s1 + distillation_loss_s3_s2)

        trainable_vars_3 = self.student3.trainable_variables
        gradients_3 = tape_student_3.gradient(loss_3, trainable_vars_3)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients_3, trainable_vars_3))

        # Update the metrics configured in `compile()`.     
        self.compiled_metrics._metrics[2].update_state(y, student_predictions_3)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.compiled_metrics._metrics}

        results.update (
            {"student_loss_1": student_loss_1,
             "student_loss_2": student_loss_2,
             "student_loss_3": student_loss_3,
             "distillation_loss_t_s1": distillation_loss_t_s1,
             "distillation_loss_s1_s2": distillation_loss_s1_s2,
             "distillation_loss_s1_s3": distillation_loss_s1_s3,
             "distillation_loss_t_s2": distillation_loss_t_s2,
             "distillation_loss_s2_s1": distillation_loss_s2_s1,
             "distillation_loss_s2_s3": distillation_loss_s2_s3,
             "distillation_loss_t_s3": distillation_loss_t_s3,
             "distillation_loss_s3_s1": distillation_loss_s3_s1,
             "distillation_loss_s3_s2": distillation_loss_s3_s2}
        )

        return results


    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction_1 = self.student1(x, training=False)
        y_prediction_2 = self.student2(x, training=False)
        y_prediction_3 = self.student3(x, training=False)

        # Calculate the loss
        student_loss_1 = self.student_loss_fn(y, y_prediction_1)
        student_loss_2 = self.student_loss_fn(y, y_prediction_2)
        student_loss_3 = self.student_loss_fn(y, y_prediction_3)

        # Update the metrics.
        self.compiled_metrics._metrics[0].update_state(y, y_prediction_1)
        self.compiled_metrics._metrics[1].update_state(y, y_prediction_2)
        self.compiled_metrics._metrics[2].update_state(y, y_prediction_3)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.compiled_metrics._metrics}

        results.update({"student_loss_1": student_loss_1,
             "student_loss_2": student_loss_2,
             "student_loss_3": student_loss_3})

        return results


In [ ]:
teacher = tf.keras.models.load_model('/saved_models/resnet50cifar')

In [ ]:
# Student 3

def get_student3():

  def preprocess_image_input(input_images):
    input_images = tf.cast(input_images, 'float32')
    output_ims = tf.keras.applications.mobilenet.preprocess_input(input_images)
    return output_ims

  class Preprocess(tf.keras.layers.Layer):
      def __init__(self):
          super(Preprocess, self).__init__()

      def call(self, inputs):
          return preprocess_image_input(inputs)

  student_mobile = tf.keras.applications.MobileNet(
      input_shape=(224, 224, 3),
      alpha=1.0,
      depth_multiplier=1,
      dropout=0.001,
      include_top=True,
      weights=None,
      input_tensor=None,
      pooling=None,
      classes=10,
      classifier_activation=None
  )

  inputs = tf.keras.layers.Input(shape=(32,32,3))
  resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)
  pre_process = Preprocess()(resize)
  resnet_extractor = student_mobile(pre_process)
  student = tf.keras.Model(inputs=inputs, outputs = resnet_extractor)
  return student

student3 = get_student3()

In [ ]:
# Student 2

def get_student2():

  # Import necessary components to build LeNet
  from keras.models import Sequential
  from keras.layers.core import Dense, Activation, Flatten
  from keras.layers.convolutional import Conv2D, MaxPooling2D
  from keras.regularizers import l2

  def lenet_model(img_shape=(32, 32, 3), n_classes=10, l2_reg=0.,
    weights=None):

    # Initialize model
    lenet = Sequential()

    # 2 sets of CRP (Convolution, RELU, Pooling)
    lenet.add(Conv2D(20, (5, 5), padding="same",
      input_shape=img_shape, kernel_regularizer=l2(l2_reg)))
    lenet.add(Activation("relu"))
    lenet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    lenet.add(Conv2D(50, (5, 5), padding="same",
      kernel_regularizer=l2(l2_reg)))
    lenet.add(Activation("relu"))
    lenet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Fully connected layers (w/ RELU)
    lenet.add(Flatten())
    lenet.add(Dense(500, kernel_regularizer=l2(l2_reg)))
    lenet.add(Activation("relu"))

    # Softmax (for classification)
    lenet.add(Dense(n_classes, kernel_regularizer=l2(l2_reg)))
    # lenet.add(Activation("softmax"))

    if weights is not None:
      lenet.load_weights(weights)

    # Return the constructed network
    return lenet

  # Create the student
  student = lenet_model()
  return student

student2 = get_student2()

In [ ]:
# Student 1

def get_student1():

  from keras.models import Sequential
  from keras.layers.core import Dense, Dropout, Activation, Flatten
  from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
  from keras.layers import BatchNormalization
  from keras.regularizers import l2

  def alexnet_model(img_shape=(32, 32, 3), n_classes=10, l2_reg=0., weights=None):

    # Initialize model
    alexnet = Sequential()

    # Layer 1
    alexnet.add(Conv2D(96, (11, 11), input_shape=img_shape,
      padding='same', kernel_regularizer=l2(l2_reg)))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

    # Layer 2
    alexnet.add(Conv2D(256, (5, 5), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

    # Layer 3
    alexnet.add(ZeroPadding2D((1, 1)))
    alexnet.add(Conv2D(512, (3, 3), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

    # Layer 4
    alexnet.add(ZeroPadding2D((1, 1)))
    alexnet.add(Conv2D(1024, (3, 3), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))

    # Layer 5
    alexnet.add(ZeroPadding2D((1, 1)))
    alexnet.add(Conv2D(1024, (3, 3), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

    # Layer 6
    alexnet.add(Flatten())
    alexnet.add(Dense(3072))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(Dropout(0.5))

    # Layer 7
    alexnet.add(Dense(4096))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(Dropout(0.5))

    # Layer 8
    alexnet.add(Dense(n_classes))

    if weights is not None:
      alexnet.load_weights(weights)

    return alexnet

  # Create the student
  student = alexnet_model()
  return student

student1 = get_student1()

In [ ]:
# Prepare the train and test dataset.
batch_size = 64

(x_train, y_train) , (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
# # x_val = x_train[0:10000]
# # y_val = y_train[0:10000]
# x_train = x_train[10000:50000]
# y_train = y_train[10000:50000]

# from sklearn.model_selection import train_test_split
# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=10000, random_state=682)

In [ ]:
# Initialize and compile distiller

beta = 0.6

print(f"alpha: 0.1; beta: {beta}; gamma: {1 - 0.1 - beta}")

student1 = get_student1()
student2 = get_student2()
student3 = get_student3()

distiller = Distiller(student1=student1, student2=student2, student3=student3, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    beta=beta,
    gamma=(1 - 0.1 - beta),
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller.evaluate(x_val, y_val)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

student1.save('/saved_models/h1_s1')
student2.save('/saved_models/h1_s2')
student3.save('/saved_models/h1_s3')

student1_test = tf.keras.models.load_model('/saved_models/h1_s1')
student2_test = tf.keras.models.load_model('/saved_models/h1_s2')
student3_test = tf.keras.models.load_model('/saved_models/h1_s3')

# Initialize and compile distiller
distiller_test = Distiller(student1=student1_test, student2=student2_test, student3=student3_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile (
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

print()
print()

alpha: 0.1; beta: 0.6; gamma: 0.30000000000000004
Epoch 1/5
938/938 [==============================] - 284s 283ms/step - s1_acc: 0.2428 - s2_acc: 0.1204 - s3_acc: 0.3289 - student_loss_1: 213.5090 - student_loss_2: 2.6233 - student_loss_3: 1.6088 - distillation_loss_t_s1: 0.0210 - distillation_loss_s1_s2: 0.0455 - distillation_loss_s1_s3: 0.0237 - distillation_loss_t_s2: 0.0280 - distillation_loss_s2_s1: 1.2780 - distillation_loss_s2_s3: 0.0184 - distillation_loss_t_s3: 0.0102 - distillation_loss_s3_s1: 1.2487 - distillation_loss_s3_s2: 0.0329
Epoch 2/5
938/938 [==============================] - 266s 283ms/step - s1_acc: 0.3339 - s2_acc: 0.1589 - s3_acc: 0.4876 - student_loss_1: 5.9992 - student_loss_2: 2.1110 - student_loss_3: 1.1614 - distillation_loss_t_s1: 0.0230 - distillation_loss_s1_s2: 0.0197 - distillation_loss_s1_s3: 0.0344 - distillation_loss_t_s2: 0.0036 - distillation_loss_s2_s1: 0.2828 - distillation_loss_s2_s3: 0.0277 - distillation_loss_t_s3: 0.0134 - distillation_loss_

In [ ]:
beta = 0.7

print(f"alpha: 0.1; beta: {beta}; gamma: {1 - 0.1 - beta}")

student1 = get_student1()
student2 = get_student2()
student3 = get_student3()

distiller = Distiller(student1=student1, student2=student2, student3=student3, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    beta=beta,
    gamma=(1 - 0.1 - beta),
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller.evaluate(x_val, y_val)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

student1.save('/saved_models/h2_s1')
student2.save('/saved_models/h2_s2')
student3.save('/saved_models/h2_s3')

student1_test = tf.keras.models.load_model('/saved_models/h2_s1')
student2_test = tf.keras.models.load_model('/saved_models/h2_s2')
student3_test = tf.keras.models.load_model('/saved_models/h2_s3')

# Initialize and compile distiller
distiller_test = Distiller(student1=student1_test, student2=student2_test, student3=student3_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

print()
print()

alpha: 0.1; beta: 0.7; gamma: 0.20000000000000007
Epoch 1/5
938/938 [==============================] - 275s 283ms/step - s1_acc: 0.2164 - s2_acc: 0.1085 - s3_acc: 0.3182 - student_loss_1: 183.0199 - student_loss_2: 3.0867 - student_loss_3: 1.6197 - distillation_loss_t_s1: 0.0212 - distillation_loss_s1_s2: 0.0604 - distillation_loss_s1_s3: 0.0252 - distillation_loss_t_s2: 0.0424 - distillation_loss_s2_s1: 1.1849 - distillation_loss_s2_s3: 0.0220 - distillation_loss_t_s3: 0.0088 - distillation_loss_s3_s1: 1.1639 - distillation_loss_s3_s2: 0.0484
Epoch 2/5
938/938 [==============================] - 265s 282ms/step - s1_acc: 0.2976 - s2_acc: 0.1043 - s3_acc: 0.4731 - student_loss_1: 3.1161 - student_loss_2: 2.2900 - student_loss_3: 1.1254 - distillation_loss_t_s1: 0.0195 - distillation_loss_s1_s2: 0.0195 - distillation_loss_s1_s3: 0.0219 - distillation_loss_t_s2: 4.9564e-04 - distillation_loss_s2_s1: 0.0843 - distillation_loss_s2_s3: 0.0172 - distillation_loss_t_s3: 0.0101 - distillation_l

In [ ]:
beta = 0.7

print(f"alpha: 0.1; beta: {beta}; gamma: {1 - 0.1 - beta}")

student1 = get_student1()
student2 = get_student2()
student3 = get_student3()

distiller = Distiller(student1=student1, student2=student2, student3=student3, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    beta=beta,
    gamma=(1 - 0.1 - beta),
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller.evaluate(x_val, y_val)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

student1.save('/saved_models/h2_s1')
student2.save('/saved_models/h2_s2')
student3.save('/saved_models/h2_s3')

student1_test = tf.keras.models.load_model('/saved_models/h2_s1')
student2_test = tf.keras.models.load_model('/saved_models/h2_s2')
student3_test = tf.keras.models.load_model('/saved_models/h2_s3')

# Initialize and compile distiller
distiller_test = Distiller(student1=student1_test, student2=student2_test, student3=student3_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

print()
print()

alpha: 0.1; beta: 0.7; gamma: 0.20000000000000007
Epoch 1/5
938/938 [==============================] - 283s 282ms/step - s1_acc: 0.1554 - s2_acc: 0.1776 - s3_acc: 0.3340 - student_loss_1: 176.3997 - student_loss_2: 2.4956 - student_loss_3: 1.5691 - distillation_loss_t_s1: 0.0229 - distillation_loss_s1_s2: 0.0431 - distillation_loss_s1_s3: 0.0281 - distillation_loss_t_s2: 0.0295 - distillation_loss_s2_s1: 0.9445 - distillation_loss_s2_s3: 0.0206 - distillation_loss_t_s3: 0.0089 - distillation_loss_s3_s1: 0.9366 - distillation_loss_s3_s2: 0.0330
Epoch 2/5
938/938 [==============================] - 263s 281ms/step - s1_acc: 0.2303 - s2_acc: 0.3011 - s3_acc: 0.4938 - student_loss_1: 3.5321 - student_loss_2: 1.6246 - student_loss_3: 1.0880 - distillation_loss_t_s1: 0.0211 - distillation_loss_s1_s2: 0.0168 - distillation_loss_s1_s3: 0.0226 - distillation_loss_t_s2: 0.0080 - distillation_loss_s2_s1: 0.1093 - distillation_loss_s2_s3: 0.0128 - distillation_loss_t_s3: 0.0111 - distillation_loss_

In [ ]:
beta = 0.8
print(f"alpha: 0.1; beta: {beta}; gamma: {1 - 0.1 - beta}")

# student1 = get_student1()
# student2 = get_student2()
# student3 = get_student3()

distiller = Distiller(student1=student1, student2=student2, student3=student3, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    beta=beta,
    gamma=(1 - 0.1 - beta),
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
# distiller.evaluate(x_val, y_val)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

student1.save('/saved_models/h3_s1')
student2.save('/saved_models/h3_s2')
student3.save('/saved_models/h3_s3')

student1_test = tf.keras.models.load_model('/saved_models/h3_s1')
student2_test = tf.keras.models.load_model('/saved_models/h3_s2')
student3_test = tf.keras.models.load_model('/saved_models/h3_s3')

# Initialize and compile distiller
distiller_test = Distiller(student1=student1_test, student2=student2_test, student3=student3_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

print()
print()

alpha: 0.1; beta: 0.8; gamma: 0.09999999999999998
Epoch 1/5
1563/1563 [==============================] - 465s 289ms/step - s1_acc: 0.2849 - s2_acc: 0.2603 - s3_acc: 0.4046 - student_loss_1: 99.9419 - student_loss_2: 2.2838 - student_loss_3: 1.3760 - distillation_loss_t_s1: 0.0211 - distillation_loss_s1_s2: 0.0478 - distillation_loss_s1_s3: 0.0286 - distillation_loss_t_s2: 0.0372 - distillation_loss_s2_s1: 0.8122 - distillation_loss_s2_s3: 0.0248 - distillation_loss_t_s3: 0.0096 - distillation_loss_s3_s1: 0.8040 - distillation_loss_s3_s2: 0.0358
Epoch 2/5
1563/1563 [==============================] - 452s 289ms/step - s1_acc: 0.4045 - s2_acc: 0.4026 - s3_acc: 0.5686 - student_loss_1: 2.1384 - student_loss_2: 1.2921 - student_loss_3: 0.8820 - distillation_loss_t_s1: 0.0201 - distillation_loss_s1_s2: 0.0155 - distillation_loss_s1_s3: 0.0191 - distillation_loss_t_s2: 0.0095 - distillation_loss_s2_s1: 0.0473 - distillation_loss_s2_s3: 0.0116 - distillation_loss_t_s3: 0.0117 - distillation_lo

In [ ]:
student1_test = tf.keras.models.load_model('/saved_models/h3_s1')
student2_test = tf.keras.models.load_model('/saved_models/h3_s2')
student3_test = tf.keras.models.load_model('/saved_models/h3_s3')

# Initialize and compile distiller
distiller_test = Distiller(student1=student1_test, student2=student2_test, student3=student3_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

print()
print()

313/313 [==============================] - 13s 38ms/step - s1_acc: 0.5521 - s2_acc: 0.5010 - s3_acc: 0.7161 - student_loss_1: 1.3726 - student_loss_2: 1.4388 - student_loss_3: 0.8551




In [ ]:
beta = 0.8
print(f"alpha: 0.1; beta: {beta}; gamma: {1 - 0.1 - beta}")

# student1 = get_student1()
# student2 = get_student2()
# student3 = get_student3()

distiller = Distiller(student1=student1, student2=student2, student3=student3, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    beta=beta,
    gamma=(1 - 0.1 - beta),
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller.evaluate(x_val, y_val)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

student1.save('/saved_models/h3_s1')
student2.save('/saved_models/h3_s2')
student3.save('/saved_models/h3_s3')

student1_test = tf.keras.models.load_model('/saved_models/h3_s1')
student2_test = tf.keras.models.load_model('/saved_models/h3_s2')
student3_test = tf.keras.models.load_model('/saved_models/h3_s3')

# Initialize and compile distiller
distiller_test = Distiller(student1=student1_test, student2=student2_test, student3=student3_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

print()
print()

alpha: 0.1; beta: 0.8; gamma: 0.09999999999999998
Epoch 1/5
1250/1250 [==============================] - 375s 289ms/step - s1_acc: 0.2614 - s2_acc: 0.1621 - s3_acc: 0.3784 - student_loss_1: 84.3195 - student_loss_2: 2.7196 - student_loss_3: 1.4489 - distillation_loss_t_s1: 0.0223 - distillation_loss_s1_s2: 0.0539 - distillation_loss_s1_s3: 0.0289 - distillation_loss_t_s2: 0.0388 - distillation_loss_s2_s1: 1.1658 - distillation_loss_s2_s3: 0.0245 - distillation_loss_t_s3: 0.0093 - distillation_loss_s3_s1: 1.1674 - distillation_loss_s3_s2: 0.0418
Epoch 2/5
1250/1250 [==============================] - 361s 289ms/step - s1_acc: 0.3698 - s2_acc: 0.2632 - s3_acc: 0.5397 - student_loss_1: 2.2843 - student_loss_2: 1.5868 - student_loss_3: 0.9627 - distillation_loss_t_s1: 0.0207 - distillation_loss_s1_s2: 0.0164 - distillation_loss_s1_s3: 0.0212 - distillation_loss_t_s2: 0.0071 - distillation_loss_s2_s1: 0.0505 - distillation_loss_s2_s3: 0.0139 - distillation_loss_t_s3: 0.0114 - distillation_lo

In [ ]:
beta = 0.8
print(f"alpha: 0.1; beta: {beta}; gamma: {1 - 0.1 - beta}")

student1 = get_student1()
student2 = get_student2()
student3 = get_student3()

distiller = Distiller(student1=student1, student2=student2, student3=student3, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    beta=beta,
    gamma=(1 - 0.1 - beta),
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller.evaluate(x_val, y_val)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

student1.save('/saved_models/h3_s1')
student2.save('/saved_models/h3_s2')
student3.save('/saved_models/h3_s3')

student1_test = tf.keras.models.load_model('/saved_models/h3_s1')
student2_test = tf.keras.models.load_model('/saved_models/h3_s2')
student3_test = tf.keras.models.load_model('/saved_models/h3_s3')

# Initialize and compile distiller
distiller_test = Distiller(student1=student1_test, student2=student2_test, student3=student3_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

print()
print()

alpha: 0.1; beta: 0.8; gamma: 0.09999999999999998
Epoch 1/5
938/938 [==============================] - 272s 280ms/step - s1_acc: 0.1506 - s2_acc: 0.1637 - s3_acc: 0.3377 - student_loss_1: 183.0129 - student_loss_2: 2.9730 - student_loss_3: 1.5900 - distillation_loss_t_s1: 0.0210 - distillation_loss_s1_s2: 0.0579 - distillation_loss_s1_s3: 0.0280 - distillation_loss_t_s2: 0.0427 - distillation_loss_s2_s1: 1.8249 - distillation_loss_s2_s3: 0.0236 - distillation_loss_t_s3: 0.0085 - distillation_loss_s3_s1: 1.8027 - distillation_loss_s3_s2: 0.0487
Epoch 2/5
938/938 [==============================] - 262s 279ms/step - s1_acc: 0.2320 - s2_acc: 0.2338 - s3_acc: 0.4865 - student_loss_1: 2.4310 - student_loss_2: 1.9011 - student_loss_3: 1.1017 - distillation_loss_t_s1: 0.0212 - distillation_loss_s1_s2: 0.0173 - distillation_loss_s1_s3: 0.0219 - distillation_loss_t_s2: 0.0043 - distillation_loss_s2_s1: 0.0439 - distillation_loss_s2_s3: 0.0138 - distillation_loss_t_s3: 0.0105 - distillation_loss_

In [ ]:
beta = 0.85

print(f"alpha: 0.1; beta: {beta}; gamma: {1 - 0.1 - beta}")

student1 = get_student1()
student2 = get_student2()
student3 = get_student3()

distiller = Distiller(student1=student1, student2=student2, student3=student3, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    beta=beta,
    gamma=(1 - 0.1 - beta),
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller.evaluate(x_val, y_val)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

student1.save('/saved_models/h4_s1')
student2.save('/saved_models/h4_s2')
student3.save('/saved_models/h4_s3')

student1_test = tf.keras.models.load_model('/saved_models/h4_s1')
student2_test = tf.keras.models.load_model('/saved_models/h4_s2')
student3_test = tf.keras.models.load_model('/saved_models/h4_s3')

# Initialize and compile distiller
distiller_test = Distiller(student1=student1_test, student2=student2_test, student3=student3_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

print()
print()

alpha: 0.1; beta: 0.85; gamma: 0.050000000000000044
Epoch 1/5
938/938 [==============================] - 274s 281ms/step - s1_acc: 0.2285 - s2_acc: 0.1443 - s3_acc: 0.3451 - student_loss_1: 196.6288 - student_loss_2: 2.6928 - student_loss_3: 1.5411 - distillation_loss_t_s1: 0.0207 - distillation_loss_s1_s2: 0.0519 - distillation_loss_s1_s3: 0.0261 - distillation_loss_t_s2: 0.0348 - distillation_loss_s2_s1: 1.3101 - distillation_loss_s2_s3: 0.0184 - distillation_loss_t_s3: 0.0083 - distillation_loss_s3_s1: 1.2978 - distillation_loss_s3_s2: 0.0397
Epoch 2/5
938/938 [==============================] - 264s 281ms/step - s1_acc: 0.3199 - s2_acc: 0.2247 - s3_acc: 0.4985 - student_loss_1: 3.1782 - student_loss_2: 1.6511 - student_loss_3: 1.0840 - distillation_loss_t_s1: 0.0211 - distillation_loss_s1_s2: 0.0171 - distillation_loss_s1_s3: 0.0230 - distillation_loss_t_s2: 0.0065 - distillation_loss_s2_s1: 0.0898 - distillation_loss_s2_s3: 0.0135 - distillation_loss_t_s3: 0.0106 - distillation_los

In [ ]:
beta = 0.85

print(f"alpha: 0.1; beta: {beta}; gamma: {1 - 0.1 - beta}")

student1 = get_student1()
student2 = get_student2()
student3 = get_student3()

distiller = Distiller(student1=student1, student2=student2, student3=student3, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    beta=beta,
    gamma=(1 - 0.1 - beta),
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller.evaluate(x_val, y_val)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

student1.save('/saved_models/h4_s1')
student2.save('/saved_models/h4_s2')
student3.save('/saved_models/h4_s3')

student1_test = tf.keras.models.load_model('/saved_models/h4_s1')
student2_test = tf.keras.models.load_model('/saved_models/h4_s2')
student3_test = tf.keras.models.load_model('/saved_models/h4_s3')

# Initialize and compile distiller
distiller_test = Distiller(student1=student1_test, student2=student2_test, student3=student3_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

print()
print()

alpha: 0.1; beta: 0.85; gamma: 0.050000000000000044
Epoch 1/5
938/938 [==============================] - 272s 280ms/step - s1_acc: 0.1653 - s2_acc: 0.1158 - s3_acc: 0.3494 - student_loss_1: 148.6810 - student_loss_2: 3.3437 - student_loss_3: 1.5249 - distillation_loss_t_s1: 0.0227 - distillation_loss_s1_s2: 0.0670 - distillation_loss_s1_s3: 0.0287 - distillation_loss_t_s2: 0.0474 - distillation_loss_s2_s1: 0.9875 - distillation_loss_s2_s3: 0.0242 - distillation_loss_t_s3: 0.0084 - distillation_loss_s3_s1: 0.9669 - distillation_loss_s3_s2: 0.0558
Epoch 2/5
938/938 [==============================] - 263s 280ms/step - s1_acc: 0.2543 - s2_acc: 0.1694 - s3_acc: 0.5028 - student_loss_1: 2.9807 - student_loss_2: 1.8834 - student_loss_3: 1.0756 - distillation_loss_t_s1: 0.0238 - distillation_loss_s1_s2: 0.0195 - distillation_loss_s1_s3: 0.0243 - distillation_loss_t_s2: 0.0050 - distillation_loss_s2_s1: 0.1025 - distillation_loss_s2_s3: 0.0146 - distillation_loss_t_s3: 0.0107 - distillation_los

In [ ]:
beta = 0.9

print(f"alpha: 0.1; beta: {beta}; gamma: {1 - 0.1 - beta}")

student1 = get_student1()
student2 = get_student2()
student3 = get_student3()

distiller = Distiller(student1=student1, student2=student2, student3=student3, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    beta=beta,
    gamma=(1 - 0.1 - beta),
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller.evaluate(x_val, y_val)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

student1.save('/saved_models/h5_s1')
student2.save('/saved_models/h5_s2')
student3.save('/saved_models/h5_s3')

student1_test = tf.keras.models.load_model('/saved_models/h5_s1')
student2_test = tf.keras.models.load_model('/saved_models/h5_s2')
student3_test = tf.keras.models.load_model('/saved_models/h5_s3')

# Initialize and compile distiller
distiller_test = Distiller(student1=student1_test, student2=student2_test, student3=student3_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

print()
print()

alpha: 0.1; beta: 0.9; gamma: 0.0
Epoch 1/5
938/938 [==============================] - 280s 284ms/step - s1_acc: 0.2508 - s2_acc: 0.1105 - s3_acc: 0.3584 - student_loss_1: 179.4278 - student_loss_2: 2.9543 - student_loss_3: 1.5237 - distillation_loss_t_s1: 0.0218 - distillation_loss_s1_s2: 0.0687 - distillation_loss_s1_s3: 0.0371 - distillation_loss_t_s2: 0.0485 - distillation_loss_s2_s1: 1.2498 - distillation_loss_s2_s3: 0.0326 - distillation_loss_t_s3: 0.0083 - distillation_loss_s3_s1: 1.2442 - distillation_loss_s3_s2: 0.0473
Epoch 2/5
938/938 [==============================] - 266s 284ms/step - s1_acc: 0.3500 - s2_acc: 0.1172 - s3_acc: 0.5033 - student_loss_1: 4.7461 - student_loss_2: 2.2265 - student_loss_3: 1.0814 - distillation_loss_t_s1: 0.0223 - distillation_loss_s1_s2: 0.0218 - distillation_loss_s1_s3: 0.0239 - distillation_loss_t_s2: 0.0011 - distillation_loss_s2_s1: 0.2255 - distillation_loss_s2_s3: 0.0176 - distillation_loss_t_s3: 0.0106 - distillation_loss_s3_s1: 0.2210 - 

In [ ]:
beta = 0.9

print(f"alpha: 0.1; beta: {beta}; gamma: {1 - 0.1 - beta}")

student1 = get_student1()
student2 = get_student2()
student3 = get_student3()

distiller = Distiller(student1=student1, student2=student2, student3=student3, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    beta=beta,
    gamma=(1 - 0.1 - beta),
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller.evaluate(x_val, y_val)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

student1.save('/saved_models/h5_s1')
student2.save('/saved_models/h5_s2')
student3.save('/saved_models/h5_s3')

student1_test = tf.keras.models.load_model('/saved_models/h5_s1')
student2_test = tf.keras.models.load_model('/saved_models/h5_s2')
student3_test = tf.keras.models.load_model('/saved_models/h5_s3')

# Initialize and compile distiller
distiller_test = Distiller(student1=student1_test, student2=student2_test, student3=student3_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

print()
print()

alpha: 0.1; beta: 0.9; gamma: 0.0
Epoch 1/5
938/938 [==============================] - 273s 280ms/step - s1_acc: 0.1524 - s2_acc: 0.2163 - s3_acc: 0.3603 - student_loss_1: 150.6048 - student_loss_2: 2.5937 - student_loss_3: 1.5220 - distillation_loss_t_s1: 0.0213 - distillation_loss_s1_s2: 0.0501 - distillation_loss_s1_s3: 0.0289 - distillation_loss_t_s2: 0.0356 - distillation_loss_s2_s1: 1.1885 - distillation_loss_s2_s3: 0.0238 - distillation_loss_t_s3: 0.0082 - distillation_loss_s3_s1: 1.1677 - distillation_loss_s3_s2: 0.0392
Epoch 2/5
938/938 [==============================] - 263s 280ms/step - s1_acc: 0.2376 - s2_acc: 0.3227 - s3_acc: 0.5035 - student_loss_1: 3.1627 - student_loss_2: 1.6043 - student_loss_3: 1.0893 - distillation_loss_t_s1: 0.0213 - distillation_loss_s1_s2: 0.0173 - distillation_loss_s1_s3: 0.0237 - distillation_loss_t_s2: 0.0073 - distillation_loss_s2_s1: 0.0885 - distillation_loss_s2_s3: 0.0137 - distillation_loss_t_s3: 0.0105 - distillation_loss_s3_s1: 0.0906 - 